<a href="https://colab.research.google.com/github/JoshuaWidjaja/IMDB_ClassificationAndPrediction/blob/updateBranch/CS175_IMDB_Classification_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The first block is for processing any imports/installations that are needed for the project, and the blocks after that are for running the code that the project uses.

In [ ]:
#Grab dataset(s) and install PyTorch
!git clone https://github.com/JoshuaWidjaja/IMDB_ClassificationAndPrediction
#I think PyTorch stays installed, but we still have to reclone the github every time.
#!pip install torch===1.7.1 torchvision===0.8.2 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install --user -U nltk

#Imports
import os


Cloning into 'IMDB_ClassificationAndPrediction'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 98523 (delta 0), reused 1 (delta 0), pack-reused 98519
Receiving objects: 100% (98523/98523), 97.87 MiB | 15.74 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Checking out files: 100% (100020/100020), done.


# Preprocessing Code 

In [ ]:
#Parse vocabulary words/weights and creates their appropriate lists. Might not be needed, text files with same information now included in the directories.

basePath = os.getcwd() + "/IMDB_ClassificationAndPrediction/IMDB_Data"
vocabFile = "/imdb.vocab"
weightFile = "/imdbEr.txt"
vocabList = []
counter = 0

#Below code retrieves every word from the vocabFile.
vocabFile = open( basePath+ vocabFile, "r", encoding= "utf-8")
try:
  for lines in vocabFile:
    if counter % 1000 == 0:
      #print("ON WORD COUNT: " + str(counter))
      pass
    vocabList.append(lines.strip("\n"))
    counter += 1
except UnicodeDecodeError:
  print("ERROR AT: " + str(counter) + " " + str(lines))

#Below code retrieves the weight of each word from the weightFile.
weightFile = open(basePath + weightFile, "r", encoding = "utf-8")
weightCounter = 0

#This dictionary maps each word with its perceived weight value
vocabWeight = dict()

for weight in weightFile:
  vocabWeight[vocabList[weightCounter]] = (float(weight.strip("\n")))
  weightCounter += 1

#Two dictionaries that have the vocabWeight sorted from greatest to lowest, and lowest to greatest respectively.
greatestWeightDict = sorted(vocabWeight.items(), key = lambda x: x[1], reverse=True)
lowestWeightDict = sorted(vocabWeight.items(), key = lambda x: x[1])
 #q
#print(vocabList)
#print(vocabWeight)
#print(greatestWeightDict)
#print(lowestWeightDict)


# Below is the script that opens the organized BOW file from the training data set. On each line of this file, it references the actual review score, then it maps wordIndex:OccurrenceCount. It starts from the positive reviews first.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import defaultdict
import os
import sys
import codecs

#Needed to adjust encoding to avoid errors when using vocabWeightDict. 
try:
    sys.stdin = codecs.getreader("utf-8")(sys.stdin.detach())
    sys.stdout = codecs.getwriter("utf-8")(sys.stdout.detach())
except:
    pass

#Paths relative to Google Colab setup
trainDataLabeledBOWFile = os.getcwd() + "/IMDB_ClassificationAndPrediction/IMDB_Data/train/labeledBow.feat"
testDataLabeledBOWFile = os.getcwd() + "/IMDB_ClassificationAndPrediction/IMDB_Data/test/labeledBow.feat"
trainDataUnlabeledBOWFILE = os.getcwd() + "/IMDB_ClassificationAndPrediction/IMDB_Data/train/unsupBow.feat"

vocabWeightFile = os.getcwd() + "/IMDB_ClassificationAndPrediction/unalteredWordWeightMap.txt"
positiveTrainSetPath = os.getcwd() + "/IMDB_ClassificationAndPrediction/IMDB_Data/train/pos"
negativeTrainSetPath = os.getcwd() + "/IMDB_ClassificationAndPrediction/IMDB_Data/train/neg"


#Open and creates a dict from existing vocabWeightFile. This is used to create and organize data regarding the reviews. 
vocabWeightDict = dict()
vocabWeightFile = open(vocabWeightFile, "r", encoding= "utf-8")
for lines in vocabWeightFile:
    word, spacer, weight = lines.split()
    vocabWeightDict[word] = weight
vocabList = list(vocabWeightDict)
vocabWeightFile.close()

#### Function for creating an InfoList
#Below syntax may be confusing. I will explain it here.
  # vocabList is the List that contains a list of all words that occur in the reivews, in the same order as given in the files. For examples vocabList[0] is "the" 
  # tempList is the mapping from the BOW that associated Word Index : Word Occurrence. For example 0 : 9 means the word at index 0 of vocabList occurs 9 times in that review.
  # Therefore, when we do vocabWeightDict[vocabList[int(tempList[0])]] we are doing the following:
    # Assuming tempList is [0, 9]. Then vocabList[int(tempList[0])] is simply vocabList[0] which is the word "the".
    # Then vocabWeightDict[vocabList[int(tempList)]] is just doing vocabWeightDict["the"] to get the corresponding weight. We then multiply this by tempList[1] which is the occurence count.
def createInfoList(infoFile):
  fileNum = 0
  documentInfoList = []
  stopWords = set(stopwords.words("english"))
  BOWContents = open(infoFile, "r", encoding="UTF-8")
  for lines in BOWContents:
    totalWeight = 0
    excludeStopWordsWeight = 0
    splitText = lines.split()
    for i in range(len(splitText)):
      if i == 0:
        pass
      else:
        tempList = splitText[i].split(":")
        totalWeight += float(vocabWeightDict[vocabList[int(tempList[0])]]) * float(tempList[1])
        if vocabList[int(tempList[0])] not in stopWords:
          excludeStopWordsWeight += float(vocabWeightDict[vocabList[int(tempList[0])]]) * float(tempList[1])
    documentInfoList.append( (fileNum, eval(splitText[0]), totalWeight, excludeStopWordsWeight))
    fileNum +=1
    #Increase to 25000 to see both positive and negative files.
    if fileNum == 25000:
      break
  BOWContents.close()
  return documentInfoList
  ##documentInfoList contains the following information in the following order: 
  #(Number of File being used, Actual Rating of the File, Weight when counting all words, Weight when excluding stop words.)

##### FUNCTION ENDS HERE #####

#Creating the documentInfoList
trainingDocumentInfoList = createInfoList(trainDataLabeledBOWFile)
testDocumentInfoList = createInfoList(testDataLabeledBOWFile)

#Uncomment to view what the document info list looks like.
print("Training InfoList: " + str(trainingDocumentInfoList))
print("Testing InfoList: " + str(testDocumentInfoList))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Training InfoList: [(0, 9, 15.92304840444889, 16.36440210426261), (1, 7, 18.218740522577196, 18.458094595937936), (2, 9, 26.029919772048096, 26.155731718221638), (3, 10, -6.0643782668713575, -1.8964950150759994), (4, 8, 33.53297732661938, 31.726516548969485), (5, 10, 32.06399022925124, 31.0593271472489), (6, 10, 11.942681986886663, 10.4846616255904), (7, 7, 19.48076990028698, 18.549785148187485), (8, 7, -0.5533551634216286, -0.7696204652655491), (9, 7, 11.771291198635963, 11.25946658042138), (10, 9, 9.193867891994161, 9.42520096072876), (11, 9, 5.471650936909679, 4.021615291956881), (12, 9, 6.996889039259679, 5.546853394306879), (13, 7, 2.773290763588222, 5.076748084133499), (14, 10, 22.930350670588428, 20.963272137683173), (15, 7, 9.588396593133611, 10.782265645731073), (16, 7, 9.687057154265718, 10.698213088120857), (17, 9, 89.27026953927047, 67.90756652570158), (18, 7, 16.6

# We can use the code here to gather information and statistics regarding the data. This will be usable primarily for the report.

In [ ]:
## Helper Function - Not used directly
#Opens files from specified file path (positive or negative) training data. Specify the directory in filePath, and the name of the file in fileName. Example function call below.
#checkReviewFile(positiveTrainSetPath, "0_9.txt") --- Will travel to the directory set in variable positiveTrainSetPath, and open file named 0_9.txt
def checkReviewFile(filePath: str, fileName: str) -> None:
    with open(os.path.join(filePath, fileName), "r") as reviewFile:
        fileContents = reviewFile.read()
#### Everything Below this line is used to check specifics of data. We will be using this for the report or for specific checkups.
## Using this part for data that might be useful on the report/training

#Variables to store results
weightGreaterWithStopWords = 0
weightLesserWithStopWords = 0
RatingWeightDictWithStopWords = defaultdict(list)

for entry in trainingDocumentInfoList:
# Uncomment below to get the rating:weight dictionary with stop words included
 RatingWeightDictWithStopWords[entry[1]].append(entry[2])
# Uncomment the below code to get the number of reviews that end up with a greater/lesser weight when stopwords are included
 if entry[2] > entry[3]:
  weightGreaterWithStopWords += 1
 else:
  weightLesserWithStopWords += 1

#This prints the number of reviews that end up with a greater weight when stopwords are included, and the number that end up with a lesser weight when stopwords are included.
print(weightGreaterWithStopWords, weightLesserWithStopWords)

#Prints the number of reviews with a specific rating and the average weight of the reviews with that rating
for keys in RatingWeightDictWithStopWords.keys():
 print("NUMBER OF REVIEWS WITH RATING " + str(keys) + ": " + str(len(RatingWeightDictWithStopWords[keys])))
 print("AVERAGE WEIGHT OF REVIEWS WITH RATING " + str(keys) +": " + str(sum(RatingWeightDictWithStopWords[keys])/len(RatingWeightDictWithStopWords[keys])))


#documentInfoList index 0 is (0, 9, 15.92304840444889, 16.36440210426261): (file number, rating, total weight with stop words, weight without stop words) 

9952 15048
NUMBER OF REVIEWS WITH RATING 9: 2263
AVERAGE WEIGHT OF REVIEWS WITH RATING 9: 25.538025335039823
NUMBER OF REVIEWS WITH RATING 7: 2496
AVERAGE WEIGHT OF REVIEWS WITH RATING 7: 18.572582186983873
NUMBER OF REVIEWS WITH RATING 10: 4732
AVERAGE WEIGHT OF REVIEWS WITH RATING 10: 19.255829873995065
NUMBER OF REVIEWS WITH RATING 8: 3009
AVERAGE WEIGHT OF REVIEWS WITH RATING 8: 23.778929775391873
NUMBER OF REVIEWS WITH RATING 3: 2420
AVERAGE WEIGHT OF REVIEWS WITH RATING 3: -11.771561434340828
NUMBER OF REVIEWS WITH RATING 1: 5100
AVERAGE WEIGHT OF REVIEWS WITH RATING 1: -22.543467375622267
NUMBER OF REVIEWS WITH RATING 4: 2696
AVERAGE WEIGHT OF REVIEWS WITH RATING 4: -4.803665397910479
NUMBER OF REVIEWS WITH RATING 2: 2284
AVERAGE WEIGHT OF REVIEWS WITH RATING 2: -16.923281383868098


# Both Perceptron/Linear Classifier and Logistic Classfier Setup Code

In [ ]:
"brief overview of activation/threshold functions: https://www.analyticsvidhya.com/blog/2020/01/fundamentals-deep-learning-activation-functions-when-to-use-them/"
"loss functions: https://medium.com/udacity-pytorch-challengers/a-brief-overview-of-loss-functions-in-pytorch-c0ddb78068f7"

'loss functions: https://medium.com/udacity-pytorch-challengers/a-brief-overview-of-loss-functions-in-pytorch-c0ddb78068f7'

In [ ]:
 # this will be perceptron/linear classifier cell
# I will be using the book "Natural Language Processing with Pytorch" 
# and the website https://medium.com/biaslyai/pytorch-introduction-to-neural-network-feedforward-neural-network-model-e7231cff47cb as a guide
# code so far is straight from the textbook
import torch.nn as nn
import torch as torch
# checking to see if work using weights, must turn to tensor
#Might want to reconsider changing to logistic classifier or do both.
class PerceptronClassifier(nn.Module):
  def __init__(self): # num_features is size of input features
    super(PerceptronClassifier, self).__init__()
    self.fcl = nn.Linear(1,1) # in_features are number of each sample size, out_features is size of output size
    self.relu = nn.ReLU() # this part is the only one i changed from the textbook, the website example used it and it made more sense to me in terms of what it outputs
    
  def forward(self, tensor ): # for now i did not put sigmoid, according to the book if we want probability then we need it
    out = self.fcl(tensor)
    out = torch.sign(out)
    out = self.relu(out) # if less than 0 returns 0, if greater returns itself. this is different from the book mostly cause i do not understand squeeze
    return out



In [ ]:
# ill do one with logisitic regression like the TA suggested
# some souces and tutorial I found will be listed below
# kaggle tutorial: https://www.kaggle.com/negation/pytorch-logistic-regression-tutorial
# website: https://towardsdatascience.com/logistic-regression-on-mnist-with-pytorch-b048327f8d19
# pytorch example on bag of words: https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html

#all three sources were pretty much the same. I decided to do the pytorch example which used log_softmax
import torch.nn.functional as functional
import torch as torch
class logisiticClassifier(nn.Module):
  def __init__(self):
    super(logisiticClassifier, self).__init__()
    self.linear = nn.Linear(1, 1)
 
  
  def forward(self, vector):
    output = functional.log_softmax(self.linear(vector), dim = 0)
    return output

# Here we set up, organize, and split the data.

In [ ]:
import torch.nn as nn
import torch as torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import math
import sklearn
from sklearn.feature_extraction.text import * 
from sklearn.model_selection import train_test_split 
from sklearn import linear_model 
from sklearn import metrics 

#Using this resource as a very rough template for what I did here: https://towardsdatascience.com/sentiment-analysis-using-lstm-step-by-step-50d074f09948

#Creating a tensor of size 25000 using information from documentInfoList
#This tensor contains the same informatoin as documentInfoList, just in tensor form.
tensor_weights_trainingDocumentInfoList_both = torch.FloatTensor(trainingDocumentInfoList) 

##### FEATURES LOCATED HERE #####
#Separates the "big" tensor three smaller ones: Ratings, weightWithStopWords, weightWithoutStopWords. 
#Cast into Numpy Array at first for what I did below, may need to change depending on what you need to do.
ratingFeature = np.asarray([tensor_weights_trainingDocumentInfoList_both[i][1] for i in range(len(tensor_weights_trainingDocumentInfoList_both))])
withStopWordsFeature = np.asarray([tensor_weights_trainingDocumentInfoList_both[i][2] for i in range(len(tensor_weights_trainingDocumentInfoList_both))])
removeStopWordsFeature =np.asarray([tensor_weights_trainingDocumentInfoList_both[i][3] for i in range(len(tensor_weights_trainingDocumentInfoList_both))])
counter = 0
sentimentRatingFeature = np.zeros(ratingFeature.shape)
sentimentRatingFeatureWithNeutral = np.zeros(ratingFeature.shape)
for i in range(len(ratingFeature)):
  if ratingFeature[i] > 5:
    sentimentRatingFeature[i] = 1
  else:
    sentimentRatingFeature[i] = 0


# in here will be sentimate rating with neutal, 2 is positive 1 is neutral and 0 is negative
##########################################################################################
for i in range(len(ratingFeature)):
  if ratingFeature[i]>7:
    sentimentRatingFeatureWithNeutral[i] = 2
  elif ratingFeature[i]< 4:
    sentimentRatingFeatureWithNeutral[i] = 0
  else:
    sentimentRatingFeatureWithNeutral[i] = 1
############################################################################################



#Right now , using 80% of training data as train and the other 20% to validate.
dataSplitFrac = 0.8
#batchSize affects how DataLoaders organize results. Example: Batchsize of 50 means will report results 50 at a time. Keep to 1 as default here.
#dataSize is just the size of the numpy array.
batchSize = 1
dataSize = len(withStopWordsFeature)

#Using SKLearn method of splitting training/validation/testing data.
#Documention of parameters located here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
trainX, validX, trainY, validY = train_test_split(withStopWordsFeature, ratingFeature, train_size = dataSplitFrac, random_state = 15)
# print('Number of training examples: ', trainX.shape[0])
# print('Number of testing examples: ',validX.shape[0])   
#Using a manual method of splitting into training and validation data for both WeightWithStopWords and Ratings.
# trainX = withStopWordsFeature[0:int(dataSize * dataSplitFrac)]
# trainY = ratingFeature[0:int(dataSize*dataSplitFrac)]

# validX = withStopWordsFeature[int(dataSize*dataSplitFrac):]
# validY = ratingFeature[int(dataSize * dataSplitFrac):]


# "Bundles" both WeightWithStopWords and Ratings into tensors 
trainData = TensorDataset(torch.from_numpy(trainX), torch.from_numpy(trainY))
validData = TensorDataset(torch.from_numpy(validX), torch.from_numpy(validY))

# Uses the trainData, which are now tensors, and shuffles them. Stores them in the DataLoader class.
trainLoader = DataLoader(trainData, shuffle=True, batch_size=batchSize)
validLoader = DataLoader(validData, shuffle=True, batch_size=batchSize)

[(0, 9, 15.92304840444889, 16.36440210426261), (1, 7, 18.218740522577196, 18.458094595937936), (2, 9, 26.029919772048096, 26.155731718221638), (3, 10, -6.0643782668713575, -1.8964950150759994), (4, 8, 33.53297732661938, 31.726516548969485), (5, 10, 32.06399022925124, 31.0593271472489), (6, 10, 11.942681986886663, 10.4846616255904), (7, 7, 19.48076990028698, 18.549785148187485), (8, 7, -0.5533551634216286, -0.7696204652655491), (9, 7, 11.771291198635963, 11.25946658042138), (10, 9, 9.193867891994161, 9.42520096072876), (11, 9, 5.471650936909679, 4.021615291956881), (12, 9, 6.996889039259679, 5.546853394306879), (13, 7, 2.773290763588222, 5.076748084133499), (14, 10, 22.930350670588428, 20.963272137683173), (15, 7, 9.588396593133611, 10.782265645731073), (16, 7, 9.687057154265718, 10.698213088120857), (17, 9, 89.27026953927047, 67.90756652570158), (18, 7, 16.632122353194465, 15.5190542829278), (19, 10, 23.883150142526205, 23.047576204508943), (20, 9, 29.839895713187033, 25.28431258744259

# Training and Testing with Perceptron Classifier

In [ ]:
# Perceptron Classifier initial

# setting up parameters and model
device = torch.device('cpu')
classifier = PerceptronClassifier()
classifier.to(device)
learning_rate = .001
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr = learning_rate)


#training the data
# training_input = torch.unsqueeze(torch.FloatTensor(trainX), 1)
# training_labels = torch.unsqueeze(torch.FloatTensor(trainY), 1)

# test_input = torch.unsqueeze(torch.FloatTensor(validX), 1)
# test_labels = torch.unsqueeze(torch.FloatTensor(validY), 1)

trainX, validX, trainY, validY = train_test_split(withStopWordsFeature, sentimentRatingFeature,
                                                  train_size = dataSplitFrac, random_state = 15)

trainData = TensorDataset(torch.from_numpy(trainX), torch.from_numpy(trainY))
validData = TensorDataset(torch.from_numpy(validX), torch.from_numpy(validY))


trainLoader = DataLoader(trainData, shuffle=True, batch_size=batchSize)
validLoader = DataLoader(validData, shuffle=True, batch_size=batchSize)



#training the data
classifier.train()

for epoch in range(2):
  for batch_index,(key, value) in enumerate(trainLoader):
      # forward
    key = key.to(device)
    value = value.to(device)
    #print(key, value)
    optimizer.zero_grad()
    output = classifier(key)
    #print(output.data, value)
    loss = criterion(output, value)
    #loss.requires_grad = True
    # backwards pass 
  
    loss.backward()
    optimizer.step()
    
    # to see for improvement of loss
    
    # if (i + 1) % 100 == 0:
    #   print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
    #         .format(epoch + 1, 2, batch_index + 1, len(trainLoader), loss.item()))

classifier.eval()


with torch.no_grad(): 
  correct = 0
  total = 0
  for images, labels in validLoader:
    labels = labels.to(device)
    outputs = classifier(images)
    predicted = output.data
    total+= labels.size(0)

    # to see predicted vs labels
    #print(predicted, labels)

    correct += (predicted == labels).sum().item()

  print('Accuracy: {} %'.format(100 * correct / total))



Accuracy: 52.14 %


# Training and Testing with Logistic Classifier

In [ ]:
# training and testing with logistic
# resource: https://medium.com/biaslyai/pytorch-linear-and-logistic-regression-models-5c5f0da2cb9
# https://towardsdatascience.com/logistic-regression-explained-593e9ddb7c6c
# setting up parameters and model
device = torch.device('cpu')
classifier = logisiticClassifier()
classifier.to(device)
learning_rate = .001
criterion = nn.L1Loss()
optimizer = torch.optim.SGD(classifier.parameters(), lr = learning_rate)
trainX, validX, trainY, validY = train_test_split(withStopWordsFeature, sentimentRatingFeature,
                                                  train_size = dataSplitFrac, random_state = 15)

trainData = TensorDataset(torch.from_numpy(trainX), torch.from_numpy(trainY))
validData = TensorDataset(torch.from_numpy(validX), torch.from_numpy(validY))

trainLoader = DataLoader(trainData, shuffle=True, batch_size=batchSize)
validLoader = DataLoader(validData, shuffle=True, batch_size=batchSize)


#training the data
classifier.train()

for epoch in range(2):
  for batch_index,(key, value) in enumerate(trainLoader):
      # forward
    key = key.to(device)
    value = value.to(device)

    optimizer.zero_grad()
    output = classifier(key)
    #print(output.data, value)
    loss = criterion(output, value)
    
    # backwards pass 
    loss.backward()
    optimizer.step()
    
    # to see if loss is improving

    # if (i + 1) % 100 == 0:
    #   print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
    #         .format(epoch + 1, 2, batch_index + 1, len(trainLoader), loss.item()))

classifier.eval()


with torch.no_grad(): 
  correct = 0
  total = 0
  for images, labels in validLoader:
    labels = labels.to(device)
    outputs = classifier(images)
    predicted = output.data
    total+= labels.size(0)

    # to see predicted vs labels
    #print(predicted, labels)
    
    correct += (predicted == labels).sum().item()

  print('Accuracy: {} %'.format(100 * correct / total))

Accuracy: 52.14 %


## **Classifiers done using Scipy with only Positive and Negative**

In [ ]:
from sklearn import linear_model
from sklearn import tree

trainX, validX, trainY, validY = train_test_split(withStopWordsFeature, sentimentRatingFeature, train_size = dataSplitFrac, random_state = 15)
print('Number of training examples: ', trainX.shape[0])
print('Number of testing examples: ',validX.shape[0]) 

Number of training examples:  20000
Number of testing examples:  5000


In [ ]:
# Perceptron with only positive and negative
print("Perceptron")
perceptron = linear_model.Perceptron()

trainX = trainX.reshape(-1,1)
validX = validX.reshape(-1,1)

# train the data 
fitPerceptronClassifier = perceptron.fit(trainX, trainY)
trainingPredictions = fitPerceptronClassifier.predict(trainX)
trainingAccuracy =  fitPerceptronClassifier.score(trainX, trainY)
print('Training accuracy:',format( 100*trainingAccuracy , '.2f') ) 
#evaluate
testPredictions = fitPerceptronClassifier.predict(validX)	 
testAccuracy = fitPerceptronClassifier.score(validX,validY)

print('Testing: accuracy:', format( 100*testAccuracy , '.2f') )

# sample_weight = [[12]] # predicts sample weight
# sample = fitPerceptronClassifier.predict(sample_weight)
# print(sample)

Perceptron
Training accuracy: 83.89
Testing: accuracy: 83.98
[1.]


In [ ]:
#decision tree with only positive and negative

decision = tree.DecisionTreeClassifier()
print("Decision Tree")

#train the data
fitdecision = decision.fit(trainX, trainY)
trainingPredictions = fitdecision.predict(trainX)
trainingAccuracy =  fitdecision.score(trainX, trainY)
print('Training accuracy Decision:',format( 100*trainingAccuracy , '.2f') ) 

#evaluate
testPredictions = fitdecision.predict(validX)	 
testAccuracy = fitdecision.score(validX,validY)

print('Testing accuracy Decision:', format( 100*testAccuracy , '.2f') )

Decision Tree
Training accuracy Decision: 100.00
Testing accuracy Decision: 77.10


# **Classifiers done with Scipy using Positive, Neutral, and Negative**

In [ ]:
# data split with neutral
# needs neutral data
# https://scikit-learn.org/stable/modules/multiclass.html
neutraltrainX, neutralvalidX, neutraltrainY, neutralvalidY = train_test_split(withStopWordsFeature,
                                                  sentimentRatingFeatureWithNeutral, train_size = dataSplitFrac, random_state = 15)
neutraltrainX = trainX.reshape(-1,1)
neutralvalidX = validX.reshape(-1,1)

In [ ]:
# logistic with positive and negative and neutral
# NEEDS NEUTRAL DATA
print("With Neutral score - logisitic")
logisticClassifierMulti = linear_model.LogisticRegression(multi_class="multinomial")


# logistic Classifier with Neutral rating
fitLogisticClassifierMulti = logisticClassifierMulti.fit(neutraltrainX, neutraltrainY)
trainingPredictions = fitLogisticClassifierMulti.predict(neutraltrainX)
trainingAccuracy =  fitLogisticClassifierMulti.score(neutraltrainX, neutraltrainY)
print('Training accuracy:',format( 100*trainingAccuracy , '.2f') ) 

#evaluate
testPredictions = fitLogisticClassifierMulti.predict(neutralvalidX)	 
testAccuracy = fitLogisticClassifierMulti.score(neutralvalidX,neutralvalidY)

print('Testing: accuracy:', format( 100*testAccuracy , '.2f') )

# for i in testPredictions:
#   print(i)



With Neutral score - logisitic
Training accuracy: 69.03
Testing: accuracy: 69.34


# Attemping alternate version of implementing logistic classification for sentiment analysis

In [ ]:
# Resource: https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression 
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

trainX, validX, trainY, validY = train_test_split(withStopWordsFeature, sentimentRatingFeature, train_size = dataSplitFrac, random_state = 15)
print('Number of training examples: ', trainX.shape[0])
print('Number of testing examples: ',validX.shape[0]) 

logisticClassifier = linear_model.LogisticRegression(penalty = "l2", fit_intercept= True)

trainX = trainX.reshape(-1,1)
validX = validX.reshape(-1,1)

fitLogisticClassifier = logisticClassifier.fit(trainX, trainY)
trainingPredictions = fitLogisticClassifier.predict(trainX)
trainingAccuracy =  fitLogisticClassifier.score(trainX, trainY)
print('Training accuracy:',format( 100*trainingAccuracy , '.2f') ) 

testPredictions = fitLogisticClassifier.predict(validX)	 
testAccuracy = fitLogisticClassifier.score(validX,validY)

print('Testing: accuracy:', format( 100*testAccuracy , '.2f') )

Number of training examples:  20000
Number of testing examples:  5000
Training accuracy: 83.89
Testing: accuracy: 83.90


# Figuring out where errors (wrong predictions) occur and why they happen for predictions that include Stop Words

In [ ]:
def getVocabWeight(word: str) -> None:
  print("The weight of word:", word, "is" , vocabWeightDict[word])

totalwrongCount = 0
falsePositives = 0
falseNegatives = 0
errorList = []
for i in range(len(validX)):
  weight = validX[i]
  #print(np.where(withStopWordsFeature == weight),weight)
  indexLocation = np.where(withStopWordsFeature == weight)[0][0]
  if sentimentRatingFeature[indexLocation] != testPredictions[i]:
    if indexLocation == 44:
      print("The weight", weight, "at location", indexLocation, "has been classified as class: ", testPredictions[i], "when it should be classified as: ", sentimentRatingFeature[indexLocation])  
    errorTuple = (indexLocation, weight)
    totalwrongCount += 1
    if testPredictions[i] == 1:
      falsePositives += 1
      errorTuple += ("False Positive",)
    else:
      falseNegatives += 1
      errorTuple += ("False Negative",)
    errorList.append(errorTuple)

print(totalwrongCount, "False Positives: ", falsePositives, "False Negatives: ", falseNegatives)
print("Error List in order of occurrence when shuffled")
print(errorList)
print("Error List sorted with respect to file it occurs in")
print(sorted(errorList, key = lambda x: x[0]))

print("Error List sorted with respect to the weight (from smallest to largest")
print(sorted(errorList, key = lambda x: x[1]))

# print("Error List sorted with respect to the weight (from largest to smallest")
# print(sorted(errorList, key = lambda x: x[1], reverse= True))
print()



  

The weight -6.5655694 at location 44 has been classified as class:  0.0 when it should be classified as:  1.0
804 False Positives:  461 False Negatives:  343
Error List in order of occurrence when shuffled
[(5861, -4.5280633, 'False Negative'), (1694, -12.314742, 'False Negative'), (11497, -8.366708, 'False Negative'), (13523, 9.048817, 'False Positive'), (12497, -3.673332, 'False Negative'), (17802, 15.962444, 'False Positive'), (13308, 37.14087, 'False Positive'), (13586, 4.5898066, 'False Positive'), (6996, -12.213581, 'False Negative'), (8546, -9.628465, 'False Negative'), (1419, -62.31891, 'False Negative'), (22333, 17.305132, 'False Positive'), (991, -11.591327, 'False Negative'), (16772, 4.314248, 'False Positive'), (11858, -3.0760176, 'False Negative'), (6990, -0.8733426, 'False Negative'), (18040, 11.222178, 'False Positive'), (11630, -4.979036, 'False Negative'), (17260, 4.8937654, 'False Positive'), (6797, -12.289931, 'False Negative'), (18690, 57.2419, 'False Positive'), (9

## A very basic experiment to see how accurate we can make our model be by multiplying the weights by abstract values and comparing them.

In [ ]:
#Results dictionary
trainLoaderResults = dict()

#Since batch size is 1, trainLoader will report one pair at a time.
#The first occurrence of a review with a certain rating has its weight multipled by .01. All further occurrences 
# of that rating has their weight multipled by .0001 and is added to the dictionary. These multiplication values are arbitrary.
for pair in trainLoader:
  if int(pair[1]) not in trainLoaderResults:
    trainLoaderResults[int(pair[1])] = pair[0] * .01
  else:
    trainLoaderResults[int(pair[1])] += (pair[0] * .0001)

#Uncomment to view what the result dictoinary looks like.
#print(sorted(trainLoaderResults.items()))

#Initializing an array of 5000 zeroes. 
correctPrediction = np.zeros(len(validLoader))

counter = 0
#For reviews in the validation dataset. If the rating of a review is > 5 and the weight is greater than the associated weight in trainLoaderResults, for this experiment
#we say the "prediction" is correct, and update the correctPrediction array by setting the index to 1.
# If the rating of a review is <5 and the weight is less than the associated weight, we also say we are correct and set the index to 1.
# Else, we set the index to 0.
for pair in validLoader:
  if int(pair[1]) > 5 and pair[0] > trainLoaderResults[int(pair[1])]:
    correctPrediction[counter] = 1
    counter += 1
  elif int(pair[1]) < 5 and pair[0] < trainLoaderResults[int(pair[1])]:
    correctPrediction[counter] = 1
    counter += 1
  else:
    #print(pair)
    correctPrediction[counter] = 0
    counter += 1

#Print our results of attemping to "predict" the validation set.
print("Percentage of Predictions Correct: " + str(sum(correctPrediction)/len(correctPrediction)* 100))
print("Amount of reviews predicted correct: " + str(sum(correctPrediction)))
print("Amount of reviews predicted incorrect: " + str(len(correctPrediction) - sum(correctPrediction)))


Percentage of Predictions Correct: 53.76
Amount of reviews predicted correct: 2688.0
Amount of reviews predicted incorrect: 2312.0


# Linear Regression for Predicting Ratings


In [ ]:
#Linear regression, sometimes will glitch and return NaN in which case it just needs to be re-run
import torch.nn
import torch.nn.functional

#Train/test tensors
train_doc = torch.FloatTensor(trainingDocumentInfoList)
test_doc = torch.FloatTensor(testDocumentInfoList)

trainX = np.asarray([train_doc[i][3] for i in range(len(train_doc))])
trainY = np.asarray([train_doc[i][1] for i in range(len(train_doc))])

testX = np.asarray([test_doc[i][3] for i in range(len(test_doc))])
testY = np.asarray([test_doc[i][1] for i in range(len(test_doc))])

#Grab total review weights for the training/validation datasets and their associated review scores
training_input = torch.unsqueeze(torch.FloatTensor(trainX), 1)
training_labels = torch.unsqueeze(torch.FloatTensor(trainY), 1)

test_input = torch.unsqueeze(torch.FloatTensor(testX), 1)
test_labels = torch.unsqueeze(torch.FloatTensor(testY), 1)

#Linear regression which takes one value (total review weight) as input and gives one value as output (rating)
class LinReg(nn.Module):
  def __init__(self):
    super(LinReg, self).__init__()
    self.hid = nn.Linear(1, 15)
    self.lin = nn.Linear(15, 1)

  def forward(self, w):
    output = torch.nn.functional.relu(self.hid(w))
    output = self.lin(output)
    return output

#Setting up the model/parameters
linreg_model = LinReg()
mse_loss = torch.nn.MSELoss()
optim = torch.optim.SGD(linreg_model.parameters(), lr = 0.001)

#Running regression
most_accurate = (None, None, None, None)

print("Running epochs...")
for i in range(30000):
  optim.zero_grad()
  predicted_labels = linreg_model(training_input)
  predicted_labels_test = linreg_model(test_input)

  test_mse_loss = torch.nn.MSELoss()(predicted_labels_test, test_labels)
  if most_accurate == (None, None, None, None):
    most_accurate = (linreg_model, i, float(test_mse_loss), float(test_mse_loss) ** 0.5)
  elif float(test_mse_loss) < most_accurate[2]:
    most_accurate = (linreg_model, i, float(test_mse_loss), float(test_mse_loss) ** 0.5)
  else:
    pass

  train_mse_loss = mse_loss(predicted_labels, training_labels)
  train_mse_loss.backward()
  optim.step()

  #print(i, float(train_mse_loss), float(test_mse_loss))
  
print("The most accurate model for the test data occurred at epoch # " + str(most_accurate[1]) + ".")
print("It had a MSE of " + str(most_accurate[2]) + ".")
print("This means that the average prediction was off by " + str(most_accurate[3]) + ".")
print("Normalized MSE: " + str(most_accurate[2] / 81.0))
print("Normalized RMSE: " + str(most_accurate[3] / 9.0))

Running epochs...


KeyboardInterrupt: ignored